In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import datetime as dt
import re
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Getting data from Web API

Set parameters

In [2]:
#https://dstock.vndirect.com.vn/lich-su-gia/TCB
ticker = 'TCB'
start_date = '2020-01-01'
end_date = dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d')
delta = dt.datetime.strptime(end_date, '%Y-%m-%d') - dt.datetime.strptime(start_date, '%Y-%m-%d')

Web API

In [3]:
price_api = f'https://finfo-api.vndirect.com.vn/v4/stock_prices?sort=date&q=code:{ticker}~date:gte:{start_date}~date:lte:{end_date}&size={delta.days+1}&page=1'

Get data API

In [4]:
HEADERS = {'User-Agent': 'Mozilla'}
res = requests.get(price_api, headers=HEADERS)
res

<Response [200]>

In [5]:
res.text[:500]

'{"data":[{"code":"TCB","date":"2022-04-08","time":"15:04:02","floor":"HOSE","type":"STOCK","basicPrice":49.3,"ceilingPrice":52.7,"floorPrice":45.85,"open":49.4,"high":49.4,"low":48.8,"close":48.85,"average":48.98,"adOpen":49.4,"adHigh":49.4,"adLow":48.8,"adClose":48.85,"adAverage":48.98,"nmVolume":7030100.0,"nmValue":3.44361145E11,"ptVolume":858000.0,"ptValue":4.22994E10,"change":-0.45,"adChange":-0.45,"pctChange":-0.9128},{"code":"TCB","date":"2022-04-07","time":"15:04:01","floor":"HOSE","type"'

In [6]:
data = res.json()['data']  
data[:500]

AttributeError: 'list' object has no attribute 'head'

Transform Json to Pandas DataFrame

In [41]:
data = pd.DataFrame(data)
data.head()

,code,date,time,floor,type,basicPrice,ceilingPrice,floorPrice,open,high,...,adLow,adClose,adAverage,nmVolume,nmValue,ptVolume,ptValue,change,adChange,pctChange
0,VND,2022-04-08,15:04:02,HOSE,STOCK,35.0,37.45,32.55,35.0,36.0,...,34.55,35.1,35.16,17989300.0,6.325200e+11,0.0,0.0,0.1,0.1,0.2857
1,VND,2022-04-07,15:04:01,HOSE,STOCK,35.5,37.95,33.05,35.5,37.0,...,35.00,35.0,36.02,23236900.0,8.369682e+11,0.0,0.0,-0.5,-0.5,-1.4085
2,VND,2022-04-06,15:04:02,HOSE,STOCK,34.6,37.00,32.20,34.0,36.0,...,34.00,35.5,35.23,18821500.0,6.630197e+11,0.0,0.0,0.9,0.9,2.6012
3,VND,2022-04-05,15:04:02,HOSE,STOCK,33.9,36.25,31.55,34.1,36.1,...,34.10,34.6,35.20,17035800.0,5.996418e+11,0.0,0.0,0.7,0.7,2.0649
4,VND,2022-04-04,15:04:02,HOSE,STOCK,31.7,33.90,29.50,32.4,33.9,...,32.35,33.9,33.55,28162500.0,9.447746e+11,0.0,0.0,2.2,2.2,6.9401


In [42]:
data.columns

Index(['code', 'date', 'time', 'floor', 'type', 'basicPrice', 'ceilingPrice',
       'floorPrice', 'open', 'high', 'low', 'close', 'average', 'adOpen',
       'adHigh', 'adLow', 'adClose', 'adAverage', 'nmVolume', 'nmValue',
       'ptVolume', 'ptValue', 'change', 'adChange', 'pctChange'],
      dtype='object')

In [43]:
data_clean = data[['code', 'date', 'floor', 'close','nmVolume', 'nmValue', 'change', 'pctChange']]
data_clean['date'] = pd.to_datetime(data_clean['date'])
data_clean.set_index('code',inplace=True)
data_clean

<ipython-input-43-90fc60bfda89>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,floor,close,nmVolume,nmValue,change,pctChange
code,,,,,,,
VND,2022-04-08,HOSE,35.10,17989300.0,6.325200e+11,0.10,0.2857
VND,2022-04-07,HOSE,35.00,23236900.0,8.369682e+11,-0.50,-1.4085
VND,2022-04-06,HOSE,35.50,18821500.0,6.630197e+11,0.90,2.6012
VND,2022-04-05,HOSE,34.60,17035800.0,5.996418e+11,0.70,2.0649
VND,2022-04-04,HOSE,33.90,28162500.0,9.447746e+11,2.20,6.9401
...,...,...,...,...,...,...,...
VND,2020-01-08,HOSE,14.10,233880.0,3.267280e+09,-0.20,-1.3986
VND,2020-01-07,HOSE,14.30,68060.0,9.709350e+08,0.00,0.0000
VND,2020-01-06,HOSE,14.30,149130.0,2.127958e+09,-0.10,-0.6944


In [44]:
data_clean.describe()

,close,nmVolume,nmValue,change,pctChange
count,560.000000,5.600000e+02,5.600000e+02,560.000000,560.000000
mean,33.853661,4.482693e+06,2.081525e+11,0.156518,0.460874
std,22.143207,4.342875e+06,2.408084e+11,1.324064,3.005631
min,10.700000,3.164000e+04,4.387795e+08,-5.000000,-9.894700
25%,13.650000,8.095550e+05,1.059096e+10,-0.262500,-1.153550
50%,28.850000,3.757900e+06,1.369207e+11,0.050000,0.173400
75%,50.600000,6.552900e+06,3.334973e+11,0.600000,1.970600
max,85.200000,3.259770e+07,1.124438e+12,5.500000,9.933800


In [45]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 560 entries, VND to VND
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       560 non-null    datetime64[ns]
 1   floor      560 non-null    object        
 2   close      560 non-null    float64       
 3   nmVolume   560 non-null    float64       
 4   nmValue    560 non-null    float64       
 5   change     560 non-null    float64       
 6   pctChange  560 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 35.0+ KB


In [46]:
data_clean.isnull().sum()

date         0
floor        0
close        0
nmVolume     0
nmValue      0
change       0
pctChange    0
dtype: int64

In [47]:
def make_graph(stock_data):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = .05) #, subplot_titles=("Historical Price", "Historical Volume")
    fig.add_trace(go.Scatter(x=data_clean.date, y=data_clean.close.astype("float"), name="Price"), row=1, col=1)
    fig.add_trace(go.Bar(x=data_clean.date, y=data_clean.nmVolume.astype("float"), name="Volume"), row=2, col=1)
#     fig.update_xaxes(title_text="Date", rangeslider= {'visible':False},row=1, col=1)
    fig.update_xaxes(title_text="Date",rangeslider= {'visible':True}, row=2, col=1)
#     fig.update_yaxes(title_text="Price", row=1, col=1)
#     fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_layout(showlegend=True,
    height=900,
    title='stock',
    xaxis_rangeslider_visible=False)
    fig.show()

In [48]:
make_graph(data_clean)

## Working with AWS S3

In [16]:
import boto3
from io import StringIO, BytesIO

In [17]:
# key_file  = pd.read_csv('D:/new_user_credentials.csv')
# ACCESS_KEY = key_file['Access key ID']
# SECRET_KEY = key_file['Secret access key']

In [18]:
# client = boto3.client(
# 's3',
# aws_access_key_id=ACCESS_KEY,
# aws_secret_access_key=SECRET_KEY)

In [27]:
bucket_name = 'stock-data-cuongnm'
# s3 = boto3.resource(
#     's3',
#     region_name='ap-southeast-1',
#     aws_access_key_id=ACCESS_KEY,
#     aws_secret_access_key=SECRET_KEY
# )
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

In [28]:
for obj in bucket.objects.all():
    print(obj.key)

TCB_stock_data_from_2020-01-01_to_2022-04-10.csv


In [21]:
print(bucket.objects.all())

s3.Bucket.objectsCollection(s3.Bucket(name='stock-data-cuongnm'), s3.ObjectSummary)


### Write file to S3

In [49]:
key = f'{ticker}_stock_data_from_{start_date}_to_{end_date}.csv' #.parquet
key

'VND_stock_data_from_2020-01-01_to_2022-04-10.csv'

In [50]:
out_buffer = StringIO() #BytesIO , StringIO
data_clean.to_csv(out_buffer) #to_parquet
bucket.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='stock-data-cuongnm', key='VND_stock_data_from_2020-01-01_to_2022-04-10.csv')

### Read file from S3

In [53]:
bucket_obj = bucket.objects.filter(Prefix = 'VND')
objects = [obj for obj in bucket_obj]
objects

[s3.ObjectSummary(bucket_name='stock-data-cuongnm', key='VND_stock_data_from_2020-01-01_to_2022-04-10.csv')]

In [55]:
for obj in bucket.objects.all():
    print(obj.key)

TCB_stock_data_from_2020-01-01_to_2022-04-10.csv
VND_stock_data_from_2020-01-01_to_2022-04-10.csv


In [56]:
csv_obj = bucket.Object(key='TCB_stock_data_from_2020-01-01_to_2022-04-10.csv').get().get('Body').read().decode('utf-8')

In [58]:
csv_obj[:100]

'code,date,floor,close,nmVolume,nmValue,change,pctChange\r\nTCB,2022-04-08,HOSE,48.85,7030100.0,3443611'

In [59]:
data = StringIO(csv_obj)
df = pd.read_csv(data,delimiter = ',')

In [61]:
df.head()

,code,date,floor,close,nmVolume,nmValue,change,pctChange
0,TCB,2022-04-08,HOSE,48.85,7030100.0,3.443611e+11,-0.45,-0.9128
1,TCB,2022-04-07,HOSE,49.30,5971000.0,2.976459e+11,-0.60,-1.2024
2,TCB,2022-04-06,HOSE,49.90,9012200.0,4.432822e+11,0.85,1.7329
3,TCB,2022-04-05,HOSE,49.05,5803000.0,2.856016e+11,-0.60,-1.2085
4,TCB,2022-04-04,HOSE,49.65,9775600.0,4.881631e+11,-0.65,-1.2922


Create .py file to schedule with AWS Lambda

In [ ]:
import pandas as pd
import requests
import datetime as dt
import re
import json

import boto3
from io import StringIO, BytesIO

def lambda_handler(event, context,ticker = 'TCB',start_date = '2020-01-01',end_date = dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d')):

    #https://dstock.vndirect.com.vn/lich-su-gia/TCB
    # ticker = 'TCB'
    # start_date = '2020-01-01'
    # end_date = dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d')
    delta = dt.datetime.strptime(end_date, '%Y-%m-%d') - dt.datetime.strptime(start_date, '%Y-%m-%d')


    price_api = f'https://finfo-api.vndirect.com.vn/v4/stock_prices?sort=date&q=code:{ticker}~date:gte:{start_date}~date:lte:{end_date}&size={delta.days+1}&page=1'

    HEADERS = {'User-Agent': 'Mozilla'}
    res = requests.get(price_api, headers=HEADERS)
    data = res.json()['data']  
    data = pd.DataFrame(data)
    data['date_clean'] = pd.to_datetime(data['date'])

    data_clean = data[['code', 'date_clean', 'floor', 'close','nmVolume', 'nmValue', 'change', 'pctChange']]
    # data_clean['date'] = pd.to_datetime(data_clean['date'])
    data_clean.set_index('code',inplace=True)

    bucket_name = 'stock-data-cuongnm'
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)

    key = f'{ticker}_stock_data_from_{start_date}_to_{end_date}.csv' #.parquet

    out_buffer = StringIO() #BytesIO , StringIO
    data_clean.to_csv(out_buffer) #to_parquet
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)

    return {
        'statusCode': 200,
        'body': json.dumps('file is created in:'+key)
    }



https://ap-southeast-1.console.aws.amazon.com/lambda/home?region=ap-southeast-1#/functions/get_stock_data?tab=code

https://ap-southeast-1.console.aws.amazon.com/events/home?region=ap-southeast-1#/rules

https://s3.console.aws.amazon.com/s3/buckets/stock-data-cuongnm?region=ap-southeast-1&tab=objects